# Google Trends Analytics for Smartphone
---
```bash
Author: Witchakorn Wanasanwongkot
Business Data Science, DataRockie School
```

### Fetch data from Google Trends

In [1]:
import pandas as pd
from pytrends.request import TrendReq
import datetime, time

# Set up timeframe
today = datetime.date.today()
first_day_of_month = datetime.date(today.year, today.month, 1)
years_ago = first_day_of_month - datetime.timedelta(days=365)
timeframe = f'{years_ago} {first_day_of_month}'
print("timeframe:", timeframe)

# Set up pytrends and fetch data for each keyword
pytrends = TrendReq()
dataframes = []
keywords = ["samsung s23", "oppo find", "oppo reno", "iphone 14", "xiaomi 13"]
for keyword in keywords:
    pytrends.build_payload(kw_list=[keyword], geo='TH', timeframe=timeframe)
    df = pytrends.interest_over_time()
    df.drop('isPartial', axis=1, inplace=True)
    df.columns = [keyword]
    dataframes.append(df)
    time.sleep(1)

# Concatenate dataframes into a single wide-format dataframe
df = pd.concat(dataframes, axis=1).reset_index()
df['date'] = df['date'].astype(str)
df.columns = [col.strip().replace(' ', '_') for col in df.columns.values.tolist()]
df

timeframe: 2022-05-01 2023-05-01


,date,samsung_s23,oppo_find,oppo_reno,iphone_14,xiaomi_13
0,2022-05-01,1,19,57,4,4
1,2022-05-08,0,17,54,4,2
2,2022-05-15,0,13,54,4,4
3,2022-05-22,1,9,57,5,0
4,2022-05-29,2,11,60,5,3
5,2022-06-05,1,10,59,6,2
6,2022-06-12,1,12,56,5,3
7,2022-06-19,1,6,70,7,3
8,2022-06-26,0,9,54,6,4
9,2022-07-03,1,10,60,7,5


### Write the final DataFrame to Google Sheets

In [143]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up Google Sheet credentials
scope = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name('gs_credentials.json', scope)
client = gspread.authorize(creds)

# Write the final dataframe to Google Sheet
sheet_name = 'Project_GoogleTrends' 
spreadsheet = client.create(sheet_name)
spreadsheet.share('witchakorn.datatalksclub@gmail.com', perm_type='user', role='writer')
worksheet = client.open(sheet_name).sheet1
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '18GeP8wGcGZMV0Wpf5n2wr547RDZ9G9ypGXIE5hvtx1o',
 'updatedRange': 'Sheet1!A1:F53',
 'updatedRows': 53,
 'updatedColumns': 6,
 'updatedCells': 318}